In [225]:
import umap
import numpy
import scipy as sp
import networkx as nx
from node2vec import Node2Vec
from pprint import pprint as pprint
import tqdm
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import seaborn
import mplcursors
import pickle
import os
%matplotlib nbagg

In [204]:
G = nx.Graph()

with open('graph.desc', 'r') as fh:
    for line_number, line in enumerate(fh):
        if line_number>=2:
            article_name = line.split("\t")[1][1:][:-2]
            line_number = int(line.split("\t")[0])
            G.add_node(line_number, article_name=article_name)
            
# for i in range(0, len(G.nodes)):
#     print(G.nodes[i])

In [205]:
with open('graph.graph', 'r') as fh:
    for line_number, edge in enumerate(fh):
        src = (edge.split("\t")[0]).split(" ")[0].rstrip()
        dest = (edge.split("\t")[0]).split(" ")[1].rstrip()
        G.add_edge(src,dest)
        # print(src, '->', dest)

In [206]:
article_name = nx.get_node_attributes(G, name='article_name')
article_name[98150]

'Miss North Carolina USA'

In [207]:
pprint(nx.info(G))

('Name: \n'
 'Type: Graph\n'
 'Number of nodes: 459513\n'
 'Number of edges: 678216\n'
 'Average degree:   2.9519')


In [224]:
min_degree = 20

nodes = [node for node, degree in G.degree() if degree<10]

In [221]:
G.remove_nodes_from(nodes)

In [222]:
G

In [223]:
article_name = nx.get_node_attributes(G, name='article_name')
article_name[98150]

'Miss North Carolina USA'

In [176]:
list(G.nodes)[:10]

['98150',
 '84681',
 '247142',
 '84680',
 '98160',
 '98159',
 '98153',
 '84740',
 '255903',
 '58593']

In [177]:
pprint(nx.info(G))

('Name: \n'
 'Type: DiGraph\n'
 'Number of nodes: 74\n'
 'Number of edges: 2691\n'
 'Average in degree:  36.3649\n'
 'Average out degree:  36.3649')


In [226]:
nodes_count = len(G.nodes)

# Precompute probabilities and generate walks
files = os.listdir(".")
if "node2vec.pickle" in files:
    with open("node2vec.pickle", "rb") as file:
        node2vec = pickle.load(file)
else:
    node2vec = Node2Vec(G, dimensions=100, walk_length=50, num_walks=500, p=1, q=1, weight_key='weight',
                 workers=8, sampling_strategy=None)

In [227]:
files = os.listdir(".")
if "node2vec.pickle" not in files:
    with open("node2vec.pickle", "wb") as file:
        pickle.dump(node2vec, file)

In [8]:
sorted(G.degree, key=lambda x: x[1], reverse=True)[:10]

[('251890', 302),
 ('251899', 260),
 ('36497', 223),
 ('122206', 206),
 ('128596', 185),
 ('228840', 179),
 ('68725', 176),
 ('53144', 163),
 ('55120', 159),
 ('79025', 158)]

In [229]:
info = nx.info(G)

In [228]:
pprint(info)

('Name: \n'
 'Type: DiGraph\n'
 'Number of nodes: 74\n'
 'Number of edges: 2691\n'
 'Average in degree:  36.3649\n'
 'Average out degree:  36.3649')


In [9]:
print(node2vec.walks[:1])

[['95194', '103030', '2153', '172172', '127596']]


In [10]:
# Embed
# model = node2vec.fit(window=10, min_count=1, batch_words=8)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)
# print(model)

NameError: name 'model' is not defined

In [ ]:
# Look for most similar nodes
# print (nx.get_node_attributes(G, "Horseland"))

# `model.wv.most_similar('55120')  # Output node names are always strings

In [ ]:
# '62737' in G

In [ ]:
# node_vectors = model.wv
# print(node_vectors)

In [12]:
# vector_1 = node_vectors['24982']
# print(vector_1)

In [ ]:
# Save embeddings for later use

# node_vectors.save_word2vec_format("art-embedding.txt")

In [13]:
# Load embeddings

from gensim.models import KeyedVectors
node_vectors_loaded = KeyedVectors.load_word2vec_format("art-embedding.txt")

In [14]:
print(node_vectors_loaded)

In [15]:
vector_2 = node_vectors_loaded['62737']
print(vector_2)

[ 4.959533   -2.3268273   0.18315113  1.1552151   3.0224924   1.0589527
  4.743352    7.194843    2.9952397  -4.7171206   0.09639875 -0.7766946
  1.4803749  -1.0916555   2.800883    2.464419   -3.899199    1.5888991
  1.377338   -1.1342841  -2.161191   -2.470918    0.25017437 -1.0007008
  1.5122728   1.3395745   8.505853   -0.56901866  1.3274035  -0.894552
 -2.6800663   5.7218676  -0.9345196   6.6233363  -2.3210645  -4.57687
 -1.2607498   5.0943303   2.0499954  -2.8422897  -3.1462235  -3.7176344
  0.9031875   1.9377841   2.5322757   0.39060575  3.3257515   1.0247849
 -0.11368857  7.8986616   5.6876006   1.3767242   7.2931066   0.6669249
  0.3028057  -5.636051   -1.9404404   2.6201584  -0.30218518  2.9206507
 -6.0342116   2.5051877  -3.0368474  -0.42255583  5.8723335  -0.73429054
  3.5434659   0.9749992   0.4697444   1.9308053  -3.0719614  -1.4419872
  2.0043864   0.1048908   5.3106112   1.0021913   4.359062   -0.10945073
 -3.5079207  -4.4435744  -0.6959874  -2.912175   -1.2804127   6.9

In [16]:
node_vectors_loaded.most_similar('62737') 

[('62740', 0.7136077284812927),
 ('49511', 0.635190486907959),
 ('55120', 0.627719521522522),
 ('121900', 0.5948958992958069),
 ('63512', 0.5508878231048584),
 ('41527', 0.531460702419281),
 ('49776', 0.5131902694702148),
 ('99871', 0.49755409359931946),
 ('167381', 0.45181402564048767),
 ('76323', 0.443172812461853)]

In [17]:
'259546' in G

True

In [150]:
G.['259546']

{}

In [36]:
wv = node_vectors_loaded.vectors

In [149]:
wv['259546']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [37]:
wv

array([[ 3.6197844e+00, -7.5986654e-01,  1.8922734e-03, ...,
        -2.2677345e+00,  3.8258708e+00, -5.2734470e+00],
       [ 4.9357586e+00,  5.2146754e+00, -4.0642247e+00, ...,
         1.7099025e+00,  3.5746083e-01, -9.6576869e-01],
       [ 4.1155868e+00,  4.5553970e+00, -3.3378363e+00, ...,
        -1.4086393e+00,  3.8253367e+00,  1.7497599e+00],
       ...,
       [-2.7919083e-03,  2.5150273e-03,  9.0601179e-04, ...,
         3.1671536e-03, -4.4957711e-03,  2.7519325e-03],
       [ 3.0416774e-03, -1.7812793e-03, -4.3925853e-03, ...,
         3.3383067e-03,  1.7243387e-03,  8.0142339e-04],
       [-4.2852694e-03,  9.5062121e-04, -4.7688065e-03, ...,
         4.7394396e-03, -2.4268874e-03,  4.5376639e-03]], dtype=float32)

In [32]:
# pprint(str(node_vectors_loaded.wv))

'<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7fa332c0f908>'


/home/grzegorz/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [146]:
def interactive_umap(n_neighbors, min_dist, n_components):
    umap_obj = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=n_components)
    transformed = umap_obj.fit_transform(wv)
    plt.figure(figsize=(7.5,7.5))
    seaborn.regplot(x=transformed[:, 0], y=transformed[:, 1], fit_reg=False)    

    plt.show()

In [148]:
interactive(interactive_umap, n_neighbors=5, min_dist=0.1, n_components=3)